In [8]:
# Example imports
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from scipy.spatial import ConvexHull

In [6]:
data = pd.read_csv('../Dataset/all_stocks_5yr.csv')
data.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [12]:
hull  = ConvexHull(data[['open', 'close']].values)  

ValueError: Points cannot contain NaN